# Identify the differences and advantages of model serving approaches: batch, realtime, and streaming


## 📌 Puntos clave

# Deploy a custom model to a model endpoint

## 📌 Puntos clave

# Use pandas to perform batch inference

## 📌 Puntos clave

# Identify how streaming inference is performed with Delta Live Tables

## 📌 Puntos clave

#  Deploy and query a model for realtime inference

## 📌 Puntos clave

# Split data between endpoints for realtime interference

## 📌 Puntos clave